In [1]:
import os
import awswrangler as wr
import pandas as pd
import numpy as np

In [2]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [3]:
# AWS directories
s3_root = 's3://crc-data-lake-research/sma/jimmy_woo'
database = 'crc_data_lake_research'
results_path = '{}/temp_resuts'.format(s3_root)
table_path = '{}/market_share/province_level/'.format(s3_root)
table_name = 'sma_market_share_province'

In [41]:
def province_query(province, tiers):
    tier_level = 'tier_2' if tiers[0][0] == '2' else 'tier_3' if tiers[0][0] == '3' else 'tier_4'
    tiers_str = '({})'.format(', '.join(["'{}'".format(str(tier)) for tier in tiers]))   
    
    query_parameters = {'province': province, 
                        'tier_level': tier_level, 
                        'tiers': tiers_str}
    query = """
    SELECT
      ':province;' as province,
      service_provider, 
      band, 
      block, 
      SUM(num_data_pts) as num_data_pts
    FROM crc_data_lake_research.sma_market_share_tier_4
    WHERE sma_market_share_tier_4.:tier_level; IN :tiers; 
    GROUP BY 1, 2, 3, 4
    ORDER BY 5 DESC
    """  

    df = wr.athena.read_sql_query(
            query,
            database=database,
            s3_output=results_path,
            keep_files=False,
            params=query_parameters
        )
    
    return df

In [42]:
def compute_market_share(df):
    total = np.sum(df['num_data_pts'])
    df['market_share'] = df['num_data_pts'] / total
        
    return df

In [44]:
# get province tier mappings
query = """
SELECT *
FROM crc_data_lake_research.province_tier_mapping
"""

province_tiers = wr.athena.read_sql_query(
                     query,
                     database=database,
                     s3_output=results_path,
                     keep_files=False,
                 )

In [47]:
err = {}

for idx, row in province_tiers.iterrows():
    province = row['province']
    tiers = row['tiers'].split(', ')

    try:
        df = province_query(province, tiers)
        df = compute_market_share(df)

        wr.s3.to_parquet(
            df,
            path=table_path,
            dataset=True,
            database=database,
            table=table_name
        )
    except Exception as e:
        err[province] = e

if err:
    err_cols = ['province', 'err_msg']
    err_df = pd.DataFrame.from_dict(data=err, orient='index').reset_index()
    err_df.columns = err_cols

    wr.s3.to_json(
        err_df,
        path='{}/market_share/province_err.json'.format(s3_root),
        dataset=False
    )

In [48]:
err

{}